# 原始数据加载（docs）

In [ ]:
import os
# 1. 设置国内镜像源
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from bertopic import BERTopic
import torch # 新增：导入PyTorch（sentence_transformers的底层）
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import joblib
import time
import json
import pandas as pd 

In [52]:
# 版本控制
version = 'V6'
start_time = 2000
end_time = 2025
data_source = 'GF专利'
model_name = "all-mpnet-base-v2"

# 全量数据集的词嵌入结果保存位置，计算耗时长，一定要保存，避免重复计算，只有数据集需要增加或变更时才需要重新计算
embeddings_path = r'results\embedding_results\embeddings_patents_zf_v4.pkl' 
# HDBSCAN网格搜索范围设置
search_sizes = [15, 20, 30, 50, 70,100,150,200,250,300]
# 当主题数量非常庞大时，可以进一步层次聚类合并主题，n_groups 是你想要的分组数量
n_groups = 50

# --- UMAP 参数设置 ---
n_neighbors = 20
umap_params = {
    "n_neighbors": n_neighbors,
    "n_components": 5,
    "min_dist": 0.1,
    "metric": 'cosine',
    "random_state": 5,
    "low_memory": True
}
# --- AI识别系统提示词设计 ---
system_prompt = """你是一名专业的国防专利与技术分析专家。
你的任务是根据提供的主題关键词，为每一个主题确定概括性名称。
要求：
1. 使用中文回答。
2. 名称格式为"编号：主题名称"，
示例如下
0: "1.武器装备结构设计与制图", 
1: "2.半导体工艺与微电子器件"

3. 输出JSON格式的字典，鍵为主题编号(数据类型为整型int)，值为主题名称(string)。
4. 主题简明，不要有多余的解释。"""

任务说明（以文献数据处理为例）：
- 1.将EXCEL数据转换为json格式，按照3TI 、1AB和2KEY WORDS，完成文本的字符串拼接，json文件中需要`year`、`title`、`abstract`、`keywords`以及`combined_text`字段；（AI应该很容易写出来相应的转换代码，该文件作为原始主题建模的原始数据文件）；
- 2.变量`docs`是模型的输入，将json文件中的`combined_text`传入到该向量中形成一个列表型数据；
- 3.根据后续代码跑通这一套BERTopic技术路线

In [39]:
# 从JSON文件加载数据
with open('data\patents_zf.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 打印最后两条数据进行检查
for item in data[-2:]:
    print(item['year'])
    #print(item['title'])
    #print(item['abstract'])
    print(item['combined_text'])


# 文本
docs = [item['combined_text'] for item in data]
timestamps = [item['year'] for item in data]

2025
Degaussing system for sanitizing data on a magnetic storage device includes a magnetic field generator, heating elements to raise device temperature before or during field application, and a thermal insulating structure that isolates the heating elements from the magnetic field generator. Degaussing system for sanitizing data on a magnetic storage device includes a magnetic field generator, heating elements to raise device temperature before or during field application, and a thermal insulating structure that isolates the heating elements from the magnetic field generator. Degaussing system for sanitizing data on a magnetic storage device includes a magnetic field generator, heating elements to raise device temperature before or during field application, and a thermal insulating structure that isolates the heating elements from the magnetic field generator. The degaussing system for sanitizing data stored in a magnetic storage device (MSD),comprises magnetic field generating eleme

In [4]:
print('共获取数据总量:', len(docs),'条')

共获取数据总量: 125510 条


# 创建模型

In [5]:
# CPU同样可以运行，只是速度较慢
# --- 核心：检查并指定设备 ---
# 检查CUDA（即NVIDIA GPU支持）是否可用
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU可用，正在使用: {torch.cuda.get_device_name(0)}")
    # 可选：如果你想使用特定的GPU（如第0块）
    # device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
    print("⚠️  未检测到可用GPU，将使用CPU。")
    import torch
print(torch.__version__) # 查看PyTorch版本
print(torch.cuda.is_available()) # 核心：查看CUDA是否可用

✅ GPU可用，正在使用: NVIDIA GeForce RTX 3050 Laptop GPU
2.6.0+cu124
True


# 预训练模型加载

In [ ]:
# 1. 统一定义路径
local_model_path = f"./my_models/{model_name}"  # 固定路径

# 2. 检查并加载模型
if not os.path.exists(local_model_path):
    print(f"模型不存在，正在下载并保存到本地: {model_name}")
    # 直接从huggingface下载
    sentence_model = SentenceTransformer(model_name)
    # 保存到本地
    sentence_model.save(local_model_path)
    print(f"模型已保存至：{local_model_path}")
else:
    print(f"模型已存在，从本地加载: {local_model_path}")
    # 从本地加载
    sentence_model = SentenceTransformer(local_model_path)

模型已存在，从本地加载: ./my_models/all-mpnet-base-v2


In [7]:
# 2. 计算并保存词向量
def compute_and_save_embeddings(docs, save_path='embeddings_test'):
    """计算并保存词向量"""
    if os.path.exists(save_path):
        print(f"加载已保存的嵌入: {save_path}")
        with open(save_path, 'rb') as f:
            embeddings = pickle.load(f)
    else:
        print("计算新的嵌入...")
        embeddings = sentence_model.encode(docs, show_progress_bar=True)
        with open(save_path, 'wb') as f:
            pickle.dump(embeddings, f)
        print(f"嵌入已保存到: {save_path}")
    return embeddings

In [ ]:
def load_stopwords(file_path):
    """从JSON文件加载停用词列表"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            stopwords = json.load(f)
        print(f"已加载 {len(stopwords['common'])} 个停用词")
        return stopwords['common']  # 转换为集合提高查找效率
    except FileNotFoundError:
        print(f"错误：文件 {file_path} 不存在")
        return set()
    except json.JSONDecodeError:
        print(f"错误：文件 {file_path} 不是有效的JSON格式")
        return set()

# # 测试示例
# stopwords = load_stopwords('data\stopwords.json')
# print(stopwords)

# 📊 BERTopic 核心模型参数设置指南

在 BERTopic 中，**UMAP（降维）** 和 **HDBSCAN（聚类）** 是决定主题质量的两个关键环节。对于 12.5 万条的大规模长文本数据，合理的参数配置能平衡“细节捕捉”与“结构稳健性”。

---

## 一、 UMAP：将高维语义压缩至低维

**主要任务：** 把 BERT 提取的几百维向量降至 5 维，同时保留文本间的相似性。

| 参数名称 | 设置建议 | 通俗理解 |
| --- | --- | --- |
| **`n_neighbors`** | **20** | **视野范围**。决定每个点看多远。值设为 20 能在保留局部细节的同时，兼顾 12.5 万条数据的整体分布，防止主题过于破碎。 |
| **`n_components`** | **5** | **保留维度**。降维后的“维度数”。设置为 5 比 2 或 3 能保留更多语义细节，避免信息在压缩过程中丢失。 |
| **`min_dist`** | **0.0** | **挤压程度**。点与点之间的最小间距。设置为 0.0 会让相似的文档尽可能“抱团”，有助于 HDBSCAN 发现更清晰的边界。 |
| **`metric`** | **'cosine'** | **相似度标准**。文本处理的黄金标准。它关注的是词汇的方向（语义），而不是文本的长短。 |
| **`random_state`** | **5** (或 42) | **结果保险单**。固定随机种子。确保你下次运行代码时，降维后的结果是一模一样的。 |

---

## 二、 HDBSCAN：在低维空间发现主题

**主要任务：** 在降维后的空间里寻找高密度区域，每个高密度区域就是一个“主题”。

| 参数名称 | 设置建议 | 通俗理解 |
| --- | --- | --- |
| **`min_cluster_size`** | **50** | **成团门槛**。定义一个主题最少要包含多少篇文档。对于 12.5 万条数据，50 是一个较细的粒度；如果觉得主题太碎，可以调大至 200-500。 |
| **`min_samples`** | **3** | **严苛程度**。定义核心点的门槛。值越小（如 3），算法越“宽容”，会将更多边缘点纳入主题；值越大，噪声（-1）会越多。 |
| **`metric`** | **'euclidean'** | **测距方式**。由于 UMAP 已经处理过余弦相似度，降维后的 5 维空间通常直接使用欧几里得距离来划分簇。 |

---

## 三、 针对“降噪”与“粒度”的调优秘籍

如果你在运行后发现效果不理想，可以参考以下逻辑进行微调：

### 1. 噪声（-1）比例太高怎么办？

* **方法 A（调松）：** 保持 `min_samples=3` 甚至降为 **1**。这会强制让更多游离的文档归入最近的主题。
* **方法 B（后处理）：** 不动聚类模型，训练后使用 `topic_model.reduce_outliers()` 强制归类。

### 2. 主题太细碎、数量太多怎么办？

* **增加 `min_cluster_size**`：从 50 提高到 **300 或 500**。这是控制主题总数最直接的杠杆。
* **后期合并**：利用 `topic_model.reduce_topics(nr_topics=100)` 手动指定最终想要的主题数。

### 3. 主题之间分不开（语义重叠）？

* **调小 `n_neighbors**`：从 20 降到 **15 或 10**。这会迫使 UMAP 更加关注局部差异，让不同的簇分得更开。

---

> **💡 提示：**
> 对于大规模数据，建议先用这套参数跑一个样本集（如 2 万条），观察主题分布。一旦确定 UMAP 效果不错，请务必**保存 UMAP 结果**（`.npy` 或 `.joblib`），以节省后续反复调参的时间成本。

# BERTopic模型创建

In [ ]:
# 你的代码流程：
# +-------------------+       +-------------------------+       +---------------+
# | 你的预计算嵌入     |------>| BERTopic.fit_transform()|------>|最终主题结果   |
# | embeddings_patents|       |                         |       |               |
# | _zf_v1.pkl        |       |                         |       | topics, probs |
# +-------------------+       +-------------------------+       +---------------+
#                                    │
#                                    ▼ 内部处理流程
#                         +-----------------------+
#                         │ 1. 接收你的 embeddings │
#                         +-----------------------+
#                                    │
#                                    ▼
#                         +-----------------------+
#                         │ 2. ✅ UMAP 降维        │
#                         +-----------------------+
#                                    │
#                                    ↓
#                         +-----------------------+
#                         │ 3. HDBSCAN 聚类        │
#                         +-----------------------+
#                                    │
#                                    ↓
#                         +----------------------------------+
#                         │ 4. Vectorizer\c-TF-IDF提取主题词等│
#                         +----------------------------------+

## 词向量嵌入

In [12]:
# 计算并保存词向量，全量词向量嵌入，建议保存，以免重复计算
embeddings = compute_and_save_embeddings(docs, embeddings_path)

加载已保存的嵌入: results\embedding_results\embeddings_patents_zf_v4.pkl


## UMAP降维

In [ ]:
# 文件路径定义
umap_cache_path = f'umap_model_{data_source}_{start_time}-{end_time}_{version}.joblib'
umap_model = UMAP(**umap_params)
# 检查模型是否存在
if os.path.exists(umap_cache_path):
    print(f"🚀 检测到已存在的降维模型，正在直接加载: {umap_cache_path}")
    umap_embeddings = joblib.load(umap_cache_path)
else:
    print("⏳ 未发现降维模型，开始执行 UMAP 降维计算（大数据量耗时较长）...")
    # 确保 umap_params 已经定义
    umap_embeddings = umap_model.fit_transform(embeddings)
    
    # 保存结果
    joblib.dump(umap_embeddings, umap_cache_path)
    print(f"✔ UMAP 降维完成并已保存至: {umap_cache_path}")

# 接下来可以直接使用 umap_embeddings 进行 HDBSCAN 聚类

🚀 检测到已存在的降维模型，正在直接加载: umap_model_GF专利_2000-2025_V6.joblib


## HDBSCAN聚类（网格搜索）

In [ ]:
best_topics = None
best_m_size = None
best_score = float('inf')

search_history = [] # 新增：用于记录网格搜索过程

print(f"{'Size':<10} | {'主题数':<8} | {'负样本数':<10} | {'噪声比例':<10} | {'耗时':<8}")
print("-" * 65)

for m_size in search_sizes:
    start_t = time.time()
    
    clusterer = HDBSCAN(
        min_cluster_size=m_size, 
        min_samples=3, 
        metric='euclidean',
        prediction_data=True 
    )
    
    labels = clusterer.fit_predict(umap_embeddings)
    
    # 计算指标
    n_outliers = (labels == -1).sum() # 负样本数
    n_topics = len(set(labels)) - (1 if -1 in labels else 0)
    outlier_perc = n_outliers / len(labels)
    duration = time.time() - start_t

    # 保存历史记录
    res = {
        "min_cluster_size": m_size,
        "n_topics": n_topics,
        "n_outliers": n_outliers,
        "outlier_perc": f"{outlier_perc:.1%}",
        "duration": f"{duration:.1f}s",
        "is_best": False
    }
    search_history.append(res)

    # 筛选逻辑：选噪声最小的
    if outlier_perc < best_score:
        best_topics = n_topics
        best_score = outlier_perc
        best_m_size = m_size
    

    print(f"{m_size:<10} | {n_topics:<10} | {n_outliers:<12} | {outlier_perc:<12.1%} | {duration:<8.1f}s")
print("-" * 65)
if best_m_size is None:
    print("未在预设主题数范围内找到参数，建议调低 min_dist 或检查 UMAP 效果")
else:
    print(f"🏆 符合条件的负样本数最少的参数值: min_cluster_size = {best_m_size} (负样本比例: {best_score:.1%},主题数：{best_topics})")

# 标记最优参数
for item in search_history:
    if item["min_cluster_size"] == best_m_size:
        item["is_best"] = True



Size       | 主题数      | 负样本数       | 噪声比例       | 耗时      
-----------------------------------------------------------------
15         | 1315       | 56965        | 45.4%        | 12.5    s
20         | 993        | 56082        | 44.7%        | 6.5     s
30         | 682        | 58736        | 46.8%        | 6.1     s
50         | 415        | 58075        | 46.3%        | 5.9     s
70         | 287        | 58273        | 46.4%        | 5.6     s
100        | 205        | 59254        | 47.2%        | 5.5     s
150        | 141        | 59144        | 47.1%        | 5.5     s
200        | 107        | 60516        | 48.2%        | 5.3     s
250        | 84         | 60575        | 48.3%        | 5.3     s
300        | 75         | 61595        | 49.1%        | 5.4     s
-----------------------------------------------------------------
🏆 符合条件的负样本数最少的参数值: min_cluster_size = 20 (负样本比例: 44.7%,主题数：993)
已加载 175 个停用词


2026-01-19 18:04:37,980 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


主题模型已保存到 results\topic_models\bertopic_patents_zf_2000-2025_V6


## 正式创建BERTopic模型

In [ ]:
## HDBSCAN聚类
best_clusterer = HDBSCAN(
    min_cluster_size=best_m_size, 
    min_samples=3, 
    metric='euclidean',
    prediction_data=True 
)

# 加载停用词
stop_words = load_stopwords('data\stopwords.json')
# 创建CountVectorizer模型,自定义停用词
vectorizer_model = CountVectorizer(
    ngram_range = (1,3), # 词组范围：1-3个词
    stop_words = stop_words, # 停用词
    min_df = 10, # 过滤低频噪声，对大样本非常重要
    max_features=100000 # 防止内存溢出
    )

# 初始化 BERTopic 时带上它
topic_model = BERTopic(
  embedding_model=sentence_model,
  vectorizer_model=vectorizer_model,
  umap_model=umap_model,       # 加上这一行
  hdbscan_model=best_clusterer # 加上这一行
)
#传入训练好的词向量，fit_transformer()包括了UMAP降维+HDBSCAN聚类，不建议这么做，可以将其拆开减少计算
#topics, probs = topic_model.fit_transform(docs, embeddings=embeddings,y = labels) 
#prob计算很花时间
topics, _ = topic_model.fit_transform(docs, embeddings=embeddings)
# 保存整个主题模型

year_range = f'{start_time}-{end_time}'
save_path = rf"results\topic_models\bertopic_{data_source}_{year_range}_{version}"
topic_model.save(save_path)
print(rf"主题模型已保存到 results\topic_models\bertopic_{data_source}_{year_range}_{version}")

In [27]:
## 直接加载主题模型
topic_model = BERTopic.load(rf"results\topic_models\bertopic_{data_source}_{year_range}_{version}")
print("初始主题模型加载成功！")

初始主题模型加载成功！


# 原始主题可视化

In [34]:
hierarchical_topics = topic_model.hierarchical_topics(docs)

100%|██████████| 992/992 [00:05<00:00, 166.77it/s]


In [37]:
# 降至二维，再可视化
reduced_embeddings = UMAP(n_neighbors=n_neighbors, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)

In [ ]:
import pandas as pd

def generate_report(umap_cfg, history, best_size, model_name, output_path):
    history_df = pd.DataFrame(history)
    html_table = history_df.to_html(classes='table table-striped', index=False)
    
    # 针对不同模型名称的简单描述（可选）
    model_desc = (
    f"本分析选用 {model_name} 模型。其核心任务是将英文专利文本转化为 768 维的高质量数字向量（Embedding）。"
    "该模型通过对技术上下文的深度理解，确保语义相近的专利文档在多维空间中具有更高的余弦相似度，"
    "为后续的聚类分析提供了高保真度的输入。"
    )
    
    html_content = f"""
    <html>
<head>
        <meta charset="utf-8">
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.3/dist/css/bootstrap.min.css">
        <style> 
            /* 关键：确保所有单元格文字水平垂直居中 */
            .table th, .table td {{ 
                text-align: center !important; 
                vertical-align: middle !important; 
            }}
            .best-row {{ 
                background-color: #d4edda !important; 
                font-weight: bold; 
                border: 2px solid #28a745 !important; 
            }} 
            .card-header {{ font-weight: bold; }}
            pre {{ background: #f8f9fa; padding: 10px; border-radius: 5px; }}
        </style>
    </head>
    </head>
    <body class="container py-5">
        <h2 class="mb-4 text-center">BERTopic主题聚类算法参数说明文档</h2>
        
        <div class="card mb-4 border-info">
            <div class="card-header bg-info text-white">1. 语义向量化配置 (Embedding)</div>
            <div class="card-body">
                <p class="card-text">
                    <b>预训练模型:</b> <code class="fs-5">{model_name}</code><br>
                    <b>模型描述:</b> {model_desc}，所使用的词嵌入模型（Pre-trained Model），在 Sentence-Transformers 家族中，它是公认的全能型选手。用于给英文文本进行词向量的嵌入。即确保“意思相近”的文档，生成的“数字序列”也相近。
                </p>
            </div>
        </div>

        <div class="card mb-4 border-primary">
            <div class="card-header bg-primary text-white">2. UMAP 降维参数配置</div>
            <div class="card-body">
                <ul class="row">
                    {"".join([f"<li class='col-md-6'><b>{k}:</b> {v}</li>" for k, v in umap_cfg.items()])}
                </ul>
            </div>
        </div>

        <div class="card mb-4 border-success">
            <div class="card-header bg-success text-white">3. HDBSCAN 网格搜索结果</div>
            <div class="card-body">
                <p>🏆 <b>最优策略:</b> 当 <code>min_cluster_size</code> 为 <b>{best_size}</b> 时，在预设主题范围内获得了最平衡的噪声比。</p>
                <div class="table-responsive">
                    {html_table.replace('<tr><td>' + str(best_size), '<tr class="best-row"><td>' + str(best_size))}
                </div>
            </div>
        </div>

        <p class="text-muted small text-end">报告自动生成于: {time.strftime('%Y-%m-%d %H:%M:%S')}</p>
    </body>
    </html>
    """
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(html_content)

# --- 0. 基础设置 ---
title_base = f'{year_range}年{data_source}数据Bertopic主题聚类结果一览表-{version}'
output_dir = f"BERTopic_Results_{year_range}_{data_source}_{version}"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- 参数说明 ---
report_name = os.path.join(output_dir, f"report_{year_range}_{data_source}_{version}.html")
generate_report(umap_params, search_history, best_m_size, model_name, report_name)
print(f"✔ 参数说明文档已生成：{report_name}")

# --- 1. 生成图表 (按照你想要的顺序) ---
#-------------------------------------------------------------------------
# [图 A] 主题关键词条形图 - 存为 barchart.html
fig_barchart = topic_model.visualize_barchart(
    top_n_topics=50, # 只预览前50个主题，若需要所有主题的关键词，则设置为best_topics
    custom_labels=True,
    n_words=10,
    height=400,
    title=f"{year_range}年{data_source}各主题关键词的c-TF-IDF权重得分条形图",
)

fig_barchart.update_layout(
    # 因为每行增加了子图，建议增加总宽度以防重叠
    width=1500, 
    # 统一字体样式
    font=dict(family="KaiTi", size=16),
    title_font=dict(family="KaiTi", size=36, color="black", weight="bold")
)
path_barchart = os.path.join(output_dir, "barchart.html")
fig_barchart.write_html(path_barchart)
#-------------------------------------------------------------------------
# [图 B] 层次聚类图 - 存为 hierarchy.html
fig_hierarchy = topic_model.visualize_hierarchy(
    hierarchical_topics=hierarchical_topics,
    custom_labels=True,
    title=f"{year_range}年{data_source}主题层次聚类图",
    height=800
)
fig_hierarchy.update_layout(
    title_x=0.5,
    # 因为每行增加了子图，建议增加总宽度以防重叠
    width=1500, 
    # 统一字体样式
    font=dict(family="KaiTi", size=16),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_hierarchy = os.path.join(output_dir, "hierarchy.html")
fig_hierarchy.write_html(path_hierarchy)
#-------------------------------------------------------------------------
# [图 C] 文档分布散点图 - 存为 documents.html
fig_documents = topic_model.visualize_documents(
    docs=[doc[:150] + "..." for doc in docs],
    reduced_embeddings=reduced_embeddings,
    custom_labels=True,
    hide_document_hover=False
)
# 更新散点图样式
fig_documents.update_layout(
    title=f"{year_range}年{data_source}主题分布图",
    title_x=0.5,
    width=1500,
    height=1200,
    margin=dict(l=80, r=80, t=100, b=80),
    font=dict(family="KaiTi", size=16, color="black"),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_documents = os.path.join(output_dir, "documents.html")
fig_documents.write_html(path_documents)
#-------------------------------------------------------------------------
# [图 B] 主题时序图 - 存为 hierarchy.html



topics_over_time = topic_model.topics_over_time(docs, timestamps, global_tuning=False, evolution_tuning=False)
fig_topic_time = topic_model.visualize_topics_over_time(
    topics_over_time,
    custom_labels=True)
# 更新散点图样式
fig_topic_time.update_layout(
    title=f"{year_range}年{data_source}主题时序图",
    title_x=0.5,
    width=1500,
    height=800,
    font=dict(family="KaiTi", size=16, color="black"),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_topic_time = os.path.join(output_dir, "topic_overtime_merged.html")
fig_topic_time.write_html(path_topic_time)
# --- 2. 创建导航索引页 (index.html) ---
# 注意：我在这里同步调整了按钮顺序和 iframe 的默认 src
index_content = f"""
<!DOCTYPE html>
<html lang="zh-CN">
<head>
    <meta charset="UTF-8">
    <title>{title_base}</title>
    <style>
        body {{ font-family: 'Microsoft YaHei', sans-serif; margin: 0; display: flex; flex-direction: column; height: 100vh; background-color: #f4f7f6; }}
        header {{ background: #2c3e50; color: white; padding: 15px 25px; display: flex; justify-content: space-between; align-items: center; box-shadow: 0 2px 5px rgba(0,0,0,0.2); }}
        h1 {{ margin: 0; font-size: 20px; }}
        nav {{ background: #ecf0f1; padding: 10px; display: flex; gap: 10px; border-bottom: 1px solid #ddd; }}
        .nav-btn {{ 
            padding: 8px 15px; background: white; border: 1px solid #bdc3c7; border-radius: 4px; 
            cursor: pointer; text-decoration: none; color: #34495e; font-size: 14px; transition: all 0.3s;
        }}
        .nav-btn:hover {{ background: #3498db; color: white; border-color: #2980b9; }}
        .nav-btn.active {{ background: #3498db; color: white; }}
        #content-frame {{ flex-grow: 1; border: none; width: 100%; }}
    </style>
</head>
<body>
    <header>
        <h1>{title_base}</h1>
        <span style="font-size: 12px; opacity: 0.8;">版本: {version}</span>
    </header>
    
    <nav>
        <a class="nav-btn" href="report_{year_range}_{data_source}_{version}.html" target="chart_frame">✨ {year_range}年{data_source}主题聚类参数说明</a>
        <a class="nav-btn" href="barchart.html" target="chart_frame">📈  {year_range}年{data_source}主题关键词权重图</a>
        <a class="nav-btn" href="hierarchy.html" target="chart_frame">📊  {year_range}年{data_source}主题层次聚类树状图</a>
        <a class="nav-btn" href="documents.html" target="chart_frame">📍  {year_range}年{data_source}主题分布散点图</a>
        <a class="nav-btn" href="topic_overtime_merged.html" target="chart_frame">⌚ {year_range}年{data_source}主题时序图</a>
    
    </nav>

    <iframe name="chart_frame" id="content-frame" src="report_{year_range}_{data_source}_{version}.html"></iframe>

    <script>
        const buttons = document.querySelectorAll('.nav-btn');
        buttons.forEach(btn => {{
            btn.addEventListener('click', function() {{
                buttons.forEach(b => b.classList.remove('active'));
                this.classList.add('active');
            }});
        }});
        // 默认高亮第一个按钮（即参数说明页面）
        buttons[0].classList.add('active');
    </script>
</body>
</html>
"""

with open(os.path.join(output_dir, f"index_{year_range}_{data_source}_res.html"), "w", encoding="utf-8") as f:
    f.write(index_content)

print(f"🎉 结果可视化已完成！已请打开文件夹: {output_dir}查看")

✔ 参数说明文档已生成：BERTopic_Results_2000-2025_GF专利_V6\report_2000-2025_GF专利_V6.html
🎉 结果可视化已完成！已请打开文件夹: BERTopic_Results_2000-2025_GF专利_V6查看


# 层次聚类

# 计算可以进一步合并的主题

In [ ]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import fcluster, cut_tree

# 1. 针对数据结构提取 Scipy 链接矩阵
def get_linkage_matrix_final(df):
    # 提取左子节点、右子节点、距离
    # 你的列名是 Child_Left_ID, Child_Right_ID, Distance
    left = df['Child_Left_ID'].values.astype(float)
    right = df['Child_Right_ID'].values.astype(float)
    dist = df['Distance'].values.astype(float)
    
    # 提取主题数量：由于你的 Topics 列存的是列表，我们需要计算每个列表的长度
    num_topics = df['Topics'].apply(len).values.astype(float)
    
    # 合并为 Scipy 需要的 Linkage Matrix (n-1, 4)
    return np.column_stack([left, right, dist, num_topics])

# 2. 执行矩阵转换
# 注意：Scipy 要求矩阵按距离从小到大排序，BERTopic 返回的顺序通常是反的（根节点在最前）
df_sorted = hierarchical_topics.sort_values('Distance')
linkage_matrix = get_linkage_matrix_final(df_sorted)

# 3. 进行分组

clusters = cut_tree(linkage_matrix, n_clusters=n_groups).flatten()

# 4. 映射回原始主题标签
# 获取所有原始主题 ID（排好序，确保与 cut_tree 结果对应）
all_topics = sorted([t for t in topic_model.get_topics().keys() if t != -1])

df_res = pd.DataFrame({
    "topic_id": all_topics,
    "group": clusters
})

# 5. 加上语义标签方便查看
topic_labels = topic_model.generate_topic_labels(nr_words=3, separator="_")
id_to_label = {int(l.split("_")[0]): l for l in topic_labels if not l.startswith("-1")}
df_res['label'] = df_res['topic_id'].map(id_to_label)

# 查看最终结果
print(df_res.head(50))

# 检查每个组包含的主题数
print("\n每组主题数量分布：")
print(df_res['group'].value_counts().sort_index())

    topic_id  group                                              label
0          0      0                             0_6c_substituted_alkyl
1          1      1                 1_magnetic resonance_mri_resonance
2          2      2                    2_heat_exchanger_heat exchanger
3          3      3                   3_monomer_polymer_polymerization
4          4      4        4_target nucleic_primer_target nucleic acid
5          5      5          5_bacterial infection_bacterial_infection
6          6      6                        6_antenna_ground plane_feed
7          7      3                 7_nanoparticle_lipid_nanoparticles
8          8      7                                    8_ion_ions_mass
9          9      5               9_hiv_hiv infection_immunodeficiency
10        10      8   10_additive manufacturing_additive_manufacturing
11        11      9                          11_traffic_vehicle_flight
12        12     10             12_nanotubes_carbon nanotubes_nanotube
13    

In [42]:
# 1. 按照 group 分组，并将 topic_id 聚合为列表
grouped_topics = df_res.groupby('group')['topic_id'].apply(list).reset_index()

# 2. 格式化输出
print(f"共识别出 {len(grouped_topics)} 个主题组：\n")

for _, row in grouped_topics.iterrows():
    # 将 group 编号加 1 方便阅读（从第一组开始计数）
    group_num = row['group'] + 1
    topic_list = row['topic_id']
    
    # 打印格式：第一组：[0, 1, 2, ...]
    print(f"第{group_num}组：{topic_list}")

共识别出 50 个主题组：

第1组：[0, 159, 195, 196, 204, 222, 281, 346, 375, 378, 427, 654, 751, 842, 901, 944, 958, 987]
第2组：[1, 13, 128, 212, 216, 412, 478, 511, 540, 572, 583, 640, 642, 709, 870, 974]
第3组：[2, 79, 120, 139, 197, 238, 267, 290, 299, 439, 548, 551, 664, 734, 741, 808, 840, 876, 929]
第4组：[3, 7, 61, 127, 136, 154, 275, 314, 322, 355, 574, 612, 614, 619, 742, 763]
第5组：[4, 131, 208, 215, 354, 401, 406, 517, 627, 674, 744, 860, 873, 960, 979]
第6组：[5, 9, 42, 101, 115, 123, 132, 140, 165, 229, 239, 241, 262, 279, 280, 284, 296, 312, 313, 320, 410, 418, 421, 429, 433, 452, 465, 491, 497, 509, 519, 523, 555, 558, 565, 596, 603, 634, 662, 682, 690, 743, 750, 754, 760, 764, 774, 777, 784, 787, 809, 854, 915, 927, 992]
第7组：[6, 168, 182, 301, 335, 405, 423, 448, 451, 531, 618, 694, 713, 729, 739, 786, 796, 807, 869, 879, 949, 983]
第8组：[8, 18, 34, 69, 137, 146, 152, 158, 174, 225, 227, 257, 298, 308, 311, 324, 352, 359, 367, 376, 398, 402, 425, 445, 449, 473, 488, 522, 539, 543, 580, 585, 587, 62

In [43]:
# 3. 如果你想同时看到这组主题的关键词标签（方便验证语义一致性）
print("\n" + "="*30 + "\n详细语义分组：\n")
for _, row in grouped_topics.iterrows():
    group_num = row['group'] + 1
    # 获取该组下所有主题的标签
    labels = df_res[df_res['group'] == row['group']]['label'].tolist()
    print(f"第{group_num}组 包含主题：")
    for lbl in labels:
        print(f"  - {lbl}")
    print("-" * 20)


详细语义分组：

第1组 包含主题：
  - 0_6c_substituted_alkyl
  - 159_androgen_androgen receptor_estrogen
  - 195_reacting_aryl_amine
  - 196_chiral_boronic_aryl
  - 204_metathesis_olefin_carbene
  - 222_porphyrin_phosphono_porphyrins
  - 281_silsesquioxane_silane_organic substrate
  - 346_isoquinoline_alkyl_substituted
  - 375_fluorophore_dye_cyanine
  - 378_quinoline_quinazoline_quinolin
  - 427_fluorous_fluoride_organic compound
  - 654_fluorescent_fluorescence_naphthalene
  - 751_platinum_platinum complex_platinum compound
  - 842_bisphosphonate_polyamine_ch ch
  - 901_topoisomerase_6c_6c alkyl
  - 944_chiral_imine_enantiomeric
  - 958_derivatives_nr_ch
  - 987_18 fluorine_labeled_iodonium
--------------------
第2组 包含主题：
  - 1_magnetic resonance_mri_resonance
  - 13_magnetic_spin_ferromagnetic
  - 128_diamond_single crystal_diamond like
  - 212_transformer_inductor_winding
  - 216_magnetometer_magnetic field_magnetic
  - 412_contrast_contrast agent_magnetic resonance
  - 478_recording_magnetic rec

In [44]:
# 1. 提取嵌套列表格式：[[组1序号], [组2序号], ...]
nested_topic_lists = grouped_topics['topic_id'].tolist()

print("--- 嵌套列表输出 ---")
print(nested_topic_lists)
print("\n" + "="*50 + "\n")

--- 嵌套列表输出 ---
[[0, 159, 195, 196, 204, 222, 281, 346, 375, 378, 427, 654, 751, 842, 901, 944, 958, 987], [1, 13, 128, 212, 216, 412, 478, 511, 540, 572, 583, 640, 642, 709, 870, 974], [2, 79, 120, 139, 197, 238, 267, 290, 299, 439, 548, 551, 664, 734, 741, 808, 840, 876, 929], [3, 7, 61, 127, 136, 154, 275, 314, 322, 355, 574, 612, 614, 619, 742, 763], [4, 131, 208, 215, 354, 401, 406, 517, 627, 674, 744, 860, 873, 960, 979], [5, 9, 42, 101, 115, 123, 132, 140, 165, 229, 239, 241, 262, 279, 280, 284, 296, 312, 313, 320, 410, 418, 421, 429, 433, 452, 465, 491, 497, 509, 519, 523, 555, 558, 565, 596, 603, 634, 662, 682, 690, 743, 750, 754, 760, 764, 774, 777, 784, 787, 809, 854, 915, 927, 992], [6, 168, 182, 301, 335, 405, 423, 448, 451, 531, 618, 694, 713, 729, 739, 786, 796, 807, 869, 879, 949, 983], [8, 18, 34, 69, 137, 146, 152, 158, 174, 225, 227, 257, 298, 308, 311, 324, 352, 359, 367, 376, 398, 402, 425, 445, 449, 473, 488, 522, 539, 543, 580, 585, 587, 626, 657, 670, 688, 697, 7

In [45]:
# 2. 合并后的主题理解方法一：提取每组的代表性关键词（丢失c-tf-idf权重）
# 逻辑：获取每组内所有主题的关键词，计算词频，取前 10 个作为组关键词
group_keywords = {}

for _, row in grouped_topics.iterrows():
    group_num = row['group']
    topics_in_group = row['topic_id']
    
    all_words_in_group = []
    for t_id in topics_in_group:
        # 获取单个主题的关键词 (返回的是 [(word, score), ...])
        words = [word for word, _ in topic_model.get_topic(t_id)]
        all_words_in_group.extend(words)
    
    # 统计词频并取前 10 个最能代表本组的词
    from collections import Counter
    # 过滤掉一些极其常见的重复词，保留每组独特的语义
    most_common_words = [word for word, count in Counter(all_words_in_group).most_common(10)]
    group_keywords[group_num + 1] = most_common_words

# 3. 格式化展示组关键词
print("--- 各组代表性关键词 ---")
for g_id, keywords in group_keywords.items():
    print(f"第{g_id}组共同特征词: {', '.join(keywords)}")

--- 各组代表性关键词 ---
第1组共同特征词: substituted, 6c, alkyl, cancer, aryl, optionally substituted, 6c alkyl, catalyst, ch, halo
第2组共同特征词: magnetic, magnetic field, field, spin, magnetic resonance, mri, resonance, resonance imaging, magnetic resonance imaging, imaging
第3组共同特征词: heat, cooling, air, fluid, vessel, exchanger, heat exchanger, heat transfer, coolant, cooling system
第4组共同特征词: nanoparticles, nanoparticle, copolymer, lipid, nanocrystals, cationic, monomer, precursor, shell, metal oxide
第5组共同特征词: primer, dna, probes, nucleic, oligonucleotide, snp, allele, genetic, alleles, amplification
第6组共同特征词: virus, vaccine, infection, antigen, immune response, immunogenic, antibody, bacterial, hepatitis, mycobacterium
第7组共同特征词: antenna, dielectric, microwave, filter, antenna system, radio, cavity, tunable, transmission line, ground
第8组共同特征词: eye, blood, sensor, plasma, radiation, plant, ice, ion, tomography, retinal
第9组共同特征词: coating, resin, aluminum, coating composition, composite, fiber, ceramic, m

# 合并主题

In [ ]:
# 合并主题后,bertopic_model会改变，如需多次合并，重新调用保存的原始模型
topic_model.merge_topics(docs, nested_topic_lists)
#topic_model.get_topic_info()

In [50]:
# 获取所有主题的字典 {topic_id: [(word, score), ...]}
all_topics_dict = topic_model.get_topics()
# 过滤掉 -1 (噪声)，并将每个词列表转为逗号连接的字符串
clean_topics_dict = {
    topic_id: ",".join([word for word, score in words_list])
    for topic_id, words_list in all_topics_dict.items()
    if topic_id != -1
}

for item in clean_topics_dict.items():
    print(item)
    
# 检查结果
print(f'已获取{len(clean_topics_dict)}个主题的详细信息，现在通过AI判断主题名称……') 

(0, 'virus,infection,amino,id,seq,seq id,sequence,antibody,acid,hiv')
(1, 'ion,system,eye,method,plasma,mass,ions,blood,device,sample')
(2, 'material,metal,coating,comprises,polymer,alloy,composite,surface,composition,wt')
(3, 'vehicle,system,aircraft,assembly,surface,underwater,first,control,actuator,structure')
(4, 'protein,amino,acid,peptide,amino acid,cell,subject,comprising,polypeptide,comprises')
(5, 'fuel,gas,catalyst,hydrogen,fuel cell,membrane,carbon,stream,metal,dioxide')
(6, 'layer,nitride,carbon,semiconductor,substrate,material,crystal,nanotubes,film,silicon')
(7, 'cancer,substituted,alkyl,6c,compound,optionally,optionally substituted,formula,new,aryl')
(8, 'light,sensor,imaging,optical,sample,analyte,layer,sensing,detector,infrared')
(9, 'cancer,subject,sample,expression,cell,cells,level,prostate,gene,tumor')
(10, 'plant,sequence,acid,gene,cell,protein,seq,seq id,id,polypeptide')
(11, 'magnetic,resonance,magnetic resonance,mri,imaging,field,image,spin,resonance imaging,mag

# AI主题识别

In [53]:
# AI识别
user_prompt = f"请分析以下主題关键词，并返回 JSON 字典：\n{clean_topics_dict}"

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import re
# 加载 .env 文件
load_dotenv(".env")
API_KEY=os.environ.get('DEEPSEEK_API_KEY')
deepseek_chat_model = "deepseek-chat" # DeepSeek-V3.2的非思考模式
client = OpenAI(
    api_key=API_KEY,
    base_url="https://api.deepseek.com")
# AI识别智能体参数设置：创建专门用于地址推理的 LLM 实例
Deepseek_reasponse = client.chat.completions.create(
    model=deepseek_chat_model,
    messages=[ # 对话消息列表
        {"role": "system", "content": system_prompt}, # 系统提示词，定义助手的行为
        {"role": "user", "content": user_prompt},
    ],
    response_format={'type': 'json_object'}, #强制json格式返回
    stream=False # 非流式响应（一次性返回完整结果）
)
content = json.loads(Deepseek_reasponse.choices[0].message.content)
#content = re.sub(r"^```json\s*|\s*```$", "", content.strip())
# 将键转为整型
formatted_labels = {int(k): v for k, v in content.items()}
print(formatted_labels)
# 使用自定义标签（需要先设置）
topic_model.set_topic_labels(formatted_labels)

{0: '1.病毒与抗体技术', 1: '2.离子与血浆分析系统', 2: '3.金属与复合材料', 3: '4.飞行器与水下航行器', 4: '5.蛋白质与多肽技术', 5: '6.燃料电池与催化剂', 6: '7.半导体与纳米材料', 7: '8.抗癌化合物合成', 8: '9.光学传感与成像', 9: '10.癌症基因表达分析', 10: '11.植物基因与蛋白质', 11: '12.磁共振成像技术', 12: '13.聚合物纳米粒子', 13: '14.神经网络与数据处理', 14: '15.热交换与流体系统', 15: '16.图像与视频处理', 16: '17.激光与光纤技术', 17: '18.计算机存储与处理', 18: '19.电力转换与能源系统', 19: '20.天线与射频技术', 20: '21.锂离子电池技术', 21: '22.核酸与DNA分析', 22: '23.燃气轮机技术', 23: '24.信号与通信电路', 24: '25.无线网络与路由', 25: '26.微流控与液滴技术', 26: '27.机器人技术与系统', 27: '28.超声与声学成像', 28: '29.场发射与显示技术', 29: '30.骨组织与支架材料', 30: '31.中子与电子束检测', 31: '32.燃料与燃烧系统', 32: '33.雷达与导航系统', 33: '34.光子与光波导技术', 34: '35.MEMS与微机械系统', 35: '36.炸药与推进剂技术', 36: '37.干细胞与培养技术', 37: '38.集成电路与逻辑设计', 38: '39.转子与航空器部件', 39: '40.心脏与microRNA分析', 40: '41.有机发光器件技术', 41: '42.神经刺激与电极阵列', 42: '43.太阳能电池技术', 43: '44.武器与弹药系统', 44: '45.钻井与地质勘探', 45: '46.听觉与语音处理', 46: '47.超导与高温超导材料', 47: '48.量子点与量子计算', 48: '49.生物质与催化转化', 49: '50.疼痛与神经药物'}


In [60]:
import pandas as pd

def generate_report(umap_cfg, history, best_size, model_name, output_path):
    history_df = pd.DataFrame(history)
    html_table = history_df.to_html(classes='table table-striped', index=False)
    
    hierarchical_desc = f"""
本分析采用了基于 <b>Scipy 链接矩阵</b> 的二叉树切分算法。首先提取 BERTopic 生成的 <code>Child_Left_ID</code> 
与 <code>Child_Right_ID</code> 建立层次关系，随后通过 <code>cut_tree</code> 算法在全局范围内进行动态切分。
该方法确保了合并过程不仅基于语义相似度，更遵循技术演进的逻辑层次，最终将原始细分主题聚类为 <b>{n_groups}</b> 个核心技术群。
"""
    # 针对不同模型名称的简单描述（可选）
    model_desc = (
    f"本分析选用 {model_name} 模型。其核心任务是将英文专利文本转化为 768 维的高质量数字向量（Embedding）。"
    "该模型通过对技术上下文的深度理解，确保语义相近的专利文档在多维空间中具有更高的余弦相似度，"
    "为后续的聚类分析提供了高保真度的输入。"
    )
    
    html_content = f"""
    <html>
<head>
        <meta charset="utf-8">
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.2.3/dist/css/bootstrap.min.css">
        <style> 
            /* 关键：确保所有单元格文字水平垂直居中 */
            .table th, .table td {{ 
                text-align: center !important; 
                vertical-align: middle !important; 
            }}
            .best-row {{ 
                background-color: #d4edda !important; 
                font-weight: bold; 
                border: 2px solid #28a745 !important; 
            }} 
            .card-header {{ font-weight: bold; }}
            pre {{ background: #f8f9fa; padding: 10px; border-radius: 5px; }}
        </style>
    </head>
    </head>
    <body class="container py-5">
        <h2 class="mb-4 text-center">BERTopic主题聚类算法参数说明文档</h2>
        
        <div class="card mb-4 border-info">
            <div class="card-header bg-info text-white">1. 语义向量化配置 (Embedding)</div>
            <div class="card-body">
                <p class="card-text">
                    <b>预训练模型:</b> <code class="fs-5">{model_name}</code><br>
                    <b>模型描述:</b> {model_desc}，所使用的词嵌入模型（Pre-trained Model），在 Sentence-Transformers 家族中，它是公认的全能型选手。用于给英文文本进行词向量的嵌入。即确保“意思相近”的文档，生成的“数字序列”也相近。
                </p>
            </div>
        </div>

        <div class="card mb-4 border-primary">
            <div class="card-header bg-primary text-white">2. UMAP 降维参数配置</div>
            <div class="card-body">
                <ul class="row">
                    {"".join([f"<li class='col-md-6'><b>{k}:</b> {v}</li>" for k, v in umap_cfg.items()])}
                </ul>
            </div>
        </div>

        <div class="card mb-4 border-success">
            <div class="card-header bg-success text-white">3. HDBSCAN 网格搜索结果</div>
            <div class="card-body">
                <p>🏆 <b>最优策略:</b> 当 <code>min_cluster_size</code> 为 <b>{best_size}</b> 时，在预设主题范围内获得了最平衡的噪声比。</p>
                <div class="table-responsive">
                    {html_table.replace('<tr><td>' + str(best_size), '<tr class="best-row"><td>' + str(best_size))}
                </div>
            </div>
        </div>

        <div class="card mb-4 border-warning">
                    <div class="card-header bg-warning text-dark">4. 主题层次合并算法 (Hierarchical Merging)</div>
                    <div class="card-body">
                        <div class="row">
                            <div class="col-md-12">
                                <p><b>算法逻辑：</b> {hierarchical_desc}</p>
                            </div>
                        </div>
                        <div class="row mt-2">
                            <div class="col-md-4">
                                <div class="p-2 border rounded bg-light text-center">
                                    <small class="text-muted d-block">合并前主题数</small>
                                    <span class="h5">{len(all_topics)}</span>
                                </div>
                            </div>
                            <div class="col-md-4">
                                <div class="p-2 border rounded bg-light text-center">
                                    <small class="text-muted d-block">目标群组数 (n_groups)</small>
                                    <span class="h5">{n_groups}</span>
                                </div>
                            </div>
                            <div class="col-md-4">
                                <div class="p-2 border rounded bg-light text-center">
                                    <small class="text-muted d-block">切分工具</small>
                                    <span class="h5">Scipy cut_tree</span>
                                </div>
                            </div>
                        </div>
                        <!-- div class="mt-4">
                            <h6>技术群组分布概览：</h6>
                            <pre><code>{df_res['group'].value_counts().sort_index().to_string()}</code></pre>
                        </div -->
                    </div>
                </div>

        <p class="text-muted small text-end">报告自动生成于: {time.strftime('%Y-%m-%d %H:%M:%S')}</p>
    </body>
    </html>
    """
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(html_content)

# --- 0. 基础设置 ---
title_base = f'{year_range}年{data_source}进一步层次聚类结果一览表-{version}'
output_dir = f"BERTopic_Results_merged{n_groups}_{year_range}_{data_source}_{version}"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- 参数说明 ---
report_name = os.path.join(output_dir, f"report_{year_range}_{data_source}_{version}.html")
generate_report(umap_params, search_history, best_m_size, model_name, report_name)
print(f"✔ 参数说明文档已生成：{report_name}")

# --- 1. 生成图表 (按照你想要的顺序) ---
#-------------------------------------------------------------------------
# [图 A] 主题关键词条形图 - 存为 barchart.html
fig_barchart = topic_model.visualize_barchart(
    top_n_topics=len(formatted_labels),
    custom_labels=True,
    n_words=10,
    height=400,
    title=f"{year_range}年{data_source}各主题关键词的c-TF-IDF权重得分条形图",
)

fig_barchart.update_layout(
    # 因为每行增加了子图，建议增加总宽度以防重叠
    width=1500, 
    # 统一字体样式
    font=dict(family="KaiTi", size=16),
    title_font=dict(family="KaiTi", size=36, color="black", weight="bold")
)
path_barchart = os.path.join(output_dir, "barchart.html")
fig_barchart.write_html(path_barchart)
#-------------------------------------------------------------------------
# [图 B] 层次聚类图 - 存为 hierarchy.html
fig_hierarchy = topic_model.visualize_hierarchy(
    hierarchical_topics=hierarchical_topics,
    custom_labels=True,
    title=f"{year_range}年{data_source}主题层次聚类图",
    height=800
)
fig_hierarchy.update_layout(
    title_x=0.5,
    # 因为每行增加了子图，建议增加总宽度以防重叠
    width=1500, 
    # 统一字体样式
    font=dict(family="KaiTi", size=16),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_hierarchy = os.path.join(output_dir, "hierarchy.html")
fig_hierarchy.write_html(path_hierarchy)
#-------------------------------------------------------------------------
# [图 C] 文档分布散点图 - 存为 documents.html
fig_documents = topic_model.visualize_documents(
    docs=[doc[:150] + "..." for doc in docs],
    reduced_embeddings=reduced_embeddings,
    custom_labels=True,
    hide_document_hover=False
)
# 更新散点图样式
fig_documents.update_layout(
    title=f"{year_range}年{data_source}主题分布图",
    title_x=0.5,
    width=1500,
    height=1200,
    margin=dict(l=50, r=250, t=100, b=50),
    font=dict(family="KaiTi", size=16, color="black"),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_documents = os.path.join(output_dir, "documents.html")
fig_documents.write_html(path_documents)
#-------------------------------------------------------------------------
# [图 D] 主题时序图 - 存为 topic_overtime_merged.html

topics_over_time = topic_model.topics_over_time(docs, timestamps, global_tuning=False, evolution_tuning=False)
fig_topic_time = topic_model.visualize_topics_over_time(
    topics_over_time,
    custom_labels=True)
# 更新散点图样式
fig_topic_time.update_layout(
    title=f"{year_range}年{data_source}主题时序图",
    title_x=0.5,
    width=1500,
    height=800,
    font=dict(family="KaiTi", size=16, color="black"),
    title_font=dict(family="KaiTi", size=30, color="black", weight="bold")
)
path_topic_time = os.path.join(output_dir, "topic_overtime_merged.html")
fig_topic_time.write_html(path_topic_time)
# --- 2. 创建导航索引页 (index.html) ---
# 注意：我在这里同步调整了按钮顺序和 iframe 的默认 src
index_content = f"""
<!DOCTYPE html>
<html lang="zh-CN">
<head>
    <meta charset="UTF-8">
    <title>{title_base}</title>
    <style>
        body {{ font-family: 'Microsoft YaHei', sans-serif; margin: 0; display: flex; flex-direction: column; height: 100vh; background-color: #f4f7f6; }}
        header {{ background: #2c3e50; color: white; padding: 15px 25px; display: flex; justify-content: space-between; align-items: center; box-shadow: 0 2px 5px rgba(0,0,0,0.2); }}
        h1 {{ margin: 0; font-size: 20px; }}
        nav {{ background: #ecf0f1; padding: 10px; display: flex; gap: 10px; border-bottom: 1px solid #ddd; }}
        .nav-btn {{ 
            padding: 8px 15px; background: white; border: 1px solid #bdc3c7; border-radius: 4px; 
            cursor: pointer; text-decoration: none; color: #34495e; font-size: 14px; transition: all 0.3s;
        }}
        .nav-btn:hover {{ background: #3498db; color: white; border-color: #2980b9; }}
        .nav-btn.active {{ background: #3498db; color: white; }}
        #content-frame {{ flex-grow: 1; border: none; width: 100%; }}
    </style>
</head>
<body>
    <header>
        <h1>{title_base}</h1>
        <span style="font-size: 12px; opacity: 0.8;">版本: {version}</span>
    </header>
    
    <nav>
        <a class="nav-btn" href="report_{year_range}_{data_source}_{version}.html" target="chart_frame">✨ {year_range}年{data_source}主题聚类参数说明</a>
        <a class="nav-btn" href="barchart.html" target="chart_frame">📈  {year_range}年{data_source}主题关键词权重图</a>
        <a class="nav-btn" href="hierarchy.html" target="chart_frame">📊  {year_range}年{data_source}主题层次聚类树状图</a>
        <a class="nav-btn" href="documents.html" target="chart_frame">📍  {year_range}年{data_source}主题分布散点图</a>
        <a class="nav-btn" href="topic_overtime_merged.html" target="chart_frame">⌚ {year_range}年{data_source}主题时序图</a>
    
    </nav>

    <iframe name="chart_frame" id="content-frame" src="report_{year_range}_{data_source}_{version}.html"></iframe>

    <script>
        const buttons = document.querySelectorAll('.nav-btn');
        buttons.forEach(btn => {{
            btn.addEventListener('click', function() {{
                buttons.forEach(b => b.classList.remove('active'));
                this.classList.add('active');
            }});
        }});
        // 默认高亮第一个按钮（即参数说明页面）
        buttons[0].classList.add('active');
    </script>
</body>
</html>
"""

with open(os.path.join(output_dir, f"index_{year_range}_{data_source}_res.html"), "w", encoding="utf-8") as f:
    f.write(index_content)

print(f"🎉 结果可视化已完成！已请打开文件夹: {output_dir}查看")

✔ 参数说明文档已生成：BERTopic_Results_merged50_2000-2025_GF专利_V6\report_2000-2025_GF专利_V6.html
🎉 结果可视化已完成！已请打开文件夹: BERTopic_Results_merged50_2000-2025_GF专利_V6查看
